In [2]:
import requests
import json
from datetime import datetime

# ==========================================
# 1. CONFIGURATION (Paste your keys again)
# ==========================================
CLIENT_ID = '0443b40a-5125-4241-b31b-b92c93122f54_a43ba522-ea21-497a-a7d0-27bc684143fe'
CLIENT_SECRET = 'DaGFcl18CyayKT9jE4Ol31H7P5RQy/Y7GrVcbqvm2QA='

# ==========================================
# 2. LOCAL DATA (Simulating NAMASTE Database)
# ==========================================
# In the real project, this comes from your database or Excel file
NAMASTE_DB = {
    "kasa": {
        "term": "Kasa",
        "english_equivalent": "Cough",
        "tm2_code": "TM2.A.021", 
        "system": "Ayurveda"
    },
    "jwara": {
        "term": "Jwara",
        "english_equivalent": "Fever",
        "tm2_code": "TM2.A.005",
        "system": "Ayurveda"
    },
    "shwasa": {
        "term": "Shwasa",
        "english_equivalent": "Asthma",
        "tm2_code": "TM2.A.055",
        "system": "Ayurveda"
    }
}

# ==========================================
# 3. HELPER FUNCTIONS (The Engine)
# ==========================================

def get_icd_token():
    """Authenticates with WHO and gets a token"""
    url = "https://icdaccessmanagement.who.int/connect/token"
    payload = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'client_credentials',
        'scope': 'icdapi_access'
    }
    try:
        response = requests.post(url, data=payload)
        response.raise_for_status()
        return response.json()['access_token']
    except Exception as e:
        print(f"Auth Error: {e}")
        return None

def search_who_api(term, token):
    """Searches WHO API for the English term"""
    # Using the specific release for better accuracy
    url = "https://id.who.int/icd/release/11/2024-01/mms/search"
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json',
        'Accept-Language': 'en',
        'API-Version': 'v2'
    }
    params = {'q': term, 'useflexisearch': 'false'} # Strict search for better mapping
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get('destinationEntities'):
            # Return the first/best result
            return data['destinationEntities'][0]
    return None

def generate_fhir_bundle(ayush_data, icd_data):
    """Creates the specific JSON format SIH judges want"""
    
    # Extract ICD details (with safety checks)
    icd_code = icd_data.get('theCode', 'Unspecified') if icd_data else 'Unknown'
    icd_title = icd_data.get('title', '').replace("<em class='found'>", "").replace("</em>", "") if icd_data else 'Unknown'
    icd_uri = icd_data.get('id', '') if icd_data else ''

    # This structure is HL7 FHIR Compliant
    fhir_response = {
        "resourceType": "Condition",
        "clinicalStatus": "active",
        "verificationStatus": "provisional",
        "category": [{
            "coding": [{
                "system": "http://terminology.hl7.org/CodeSystem/condition-category",
                "code": "encounter-diagnosis",
                "display": "Encounter Diagnosis"
            }]
        }],
        "code": {
            "coding": [
                # 1. The AYUSH Code (From NAMASTE)
                {
                    "system": "http://namaste.ayush.gov.in/tm2",
                    "code": ayush_data['tm2_code'],
                    "display": f"{ayush_data['term']} ({ayush_data['system']})"
                },
                # 2. The Modern Medicine Code (From WHO API)
                {
                    "system": "http://id.who.int/icd/release/11/mms",
                    "code": icd_code,
                    "display": icd_title,
                    "userSelected": False
                }
            ],
            "text": f"{ayush_data['term']} mapped to {icd_title}"
        },
        "subject": {
            "reference": "Patient/example",
            "display": "Anonymous Patient"
        },
        "recordedDate": datetime.now().isoformat()
    }
    return fhir_response

# ==========================================
# 4. MAIN EXECUTION LOOP
# ==========================================
if __name__ == "__main__":
    print("--- 🏥 SIH 2024 Integration Prototype ---")
    
    # 1. Get Token
    token = get_icd_token()
    if not token:
        exit()

    while True:
        # 2. User Input (Simulating Doctor)
        print("\nEnter an Ayurveda Symptom (e.g., 'kasa', 'jwara', 'shwasa') or 'exit':")
        user_input = input("> ").lower().strip()
        
        if user_input == 'exit':
            break
            
        # 3. Look up in Local DB (NAMASTE)
        if user_input in NAMASTE_DB:
            local_data = NAMASTE_DB[user_input]
            print(f"✅ Found in NAMASTE: {local_data['term']} -> English: {local_data['english_equivalent']}")
            
            # 4. Fetch from WHO API
            print("⏳ Fetching ICD-11 Code from WHO...")
            icd_result = search_who_api(local_data['english_equivalent'], token)
            
            # 5. Generate Standard Output
            fhir_output = generate_fhir_bundle(local_data, icd_result)
            
            print("\n--- 📄 FINAL EMR-COMPLIANT OUTPUT ---")
            print(json.dumps(fhir_output, indent=2))
            print("-------------------------------------")
            
        else:
            print("❌ Term not found in local Ayurveda database. Try 'kasa' or 'jwara'.")

--- 🏥 SIH 2024 Integration Prototype ---

Enter an Ayurveda Symptom (e.g., 'kasa', 'jwara', 'shwasa') or 'exit':


>  shwasa


✅ Found in NAMASTE: Shwasa -> English: Asthma
⏳ Fetching ICD-11 Code from WHO...

--- 📄 FINAL EMR-COMPLIANT OUTPUT ---
{
  "resourceType": "Condition",
  "clinicalStatus": "active",
  "verificationStatus": "provisional",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/condition-category",
          "code": "encounter-diagnosis",
          "display": "Encounter Diagnosis"
        }
      ]
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://namaste.ayush.gov.in/tm2",
        "code": "TM2.A.055",
        "display": "Shwasa (Ayurveda)"
      },
      {
        "system": "http://id.who.int/icd/release/11/mms",
        "code": "CA23.32",
        "display": "Unspecified asthma, uncomplicated",
        "userSelected": false
      }
    ],
    "text": "Shwasa mapped to Unspecified asthma, uncomplicated"
  },
  "subject": {
    "reference": "Patient/example",
    "display": "Anonymous Patient"
  },
  "recordedDate

>  exit
